In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   409   0  0   1126   1133  1135  0.01  0.33  0.38  0.79  0.71
1   486   1  0   1098   1105  1106  0.01  0.38  0.41  0.89  0.80
2   485   2  0   1078   1087  1086  0.01  0.37  0.40  0.87  0.77
3   508   3  0   1062   1074  1071  0.01  0.38  0.40  0.86  0.78
4   504   4  0   1048   1061  1057  0.01  0.38  0.40  0.85  0.77
5   510   5  0   1036   1051  1046  0.01  0.38  0.41  0.84  0.76
6   509   6  0   1026   1042  1036  0.01  0.38  0.41  0.82  0.75
7   502   7  0   1016   1032  1028  0.01  0.38  0.41  0.81  0.74
8   516   8  0   1007   1023  1020  0.01  0.38  0.42  0.80  0.74
9   522   9  0    999   1015  1013  0.01  0.38  0.42  0.80  0.73
10  529  10  0    990   1007  1006  0.01  0.38  0.42  0.79  0.73
11  531  11  0    983    999   999  0.01  0.38  0.43  0.78  0.73
12  524  12  0    977    993   992  0.01  0.38  0.43  0.78  0.73
13  515  13  0    971    987   986  0.01  0.38  0.43  0.78  0.72
14  510  14  0    964    982   980  0.01  0.38  0.43  0.77  0.72
15  512  15  0    959    977   975  0.01  0.38  0.43  0.77  0.72
16  513  16  0    953    972   970  0.01  0.38  0.44  0.76  0.72
17  504  17  0    948    968   964  0.01  0.38  0.44  0.76  0.72
18  505  18  0    943    962   957  0.02  0.37  0.44  0.76  0.72
19  508  19  0    939    956   951  0.02  0.37  0.44  0.75  0.72
20  534  20  0    935    950   946  0.02  0.38  0.44  0.78  0.75
21  505  21  0    932    946   942  0.02  0.37  0.44  0.75  0.71
22  509  22  0    929    941   938  0.02  0.38  0.45  0.74  0.70
23  524  23  0    925    938   935  0.02  0.39  0.45  0.77  0.73
24  524  24  0    922    934   931  0.02  0.39  0.45  0.76  0.73
25  528  25  0    916    931   928  0.02  0.39  0.45  0.75  0.71
26  526  26  0    912    926   924  0.02  0.40  0.45  0.73  0.70
27  524  27  0    908    922   922  0.02  0.40  0.45  0.72  0.69
28  529  28  0    906    918   919  0.02  0.40  0.45  0.72  0.69
29  533  29  0    903    915   917  0.03  0.40  0.45  0.72  0.69
30  535  30  0    897    911   913  0.03  0.40  0.45  0.71  0.69
31  537  31  0    892    909   909  0.03  0.40  0.45  0.71  0.68
32  537  32  0    884    905   905  0.03  0.40  0.46  0.69  0.67
33  544  33  0    878    901   901  0.03  0.40  0.47  0.66  0.65
34  544  34  0    874    899   898  0.03  0.40  0.47  0.66  0.65
35  541  35  0    872    897   895  0.03  0.40  0.47  0.65  0.65
36  553  36  0    870    894   892  0.03  0.40  0.47  0.65  0.65
37  556  37  0    868    892   889  0.03  0.40  0.47  0.65  0.65
38  635  38  0    866    890   888  0.03  0.40  0.47  0.65  0.65
39  663  39  0    864    887   886  0.04  0.40  0.47  0.65  0.65
40  661  40  0    863    886   885  0.04  0.40  0.47  0.65  0.65
41  661  41  0    862    885   883  0.04  0.39  0.47  0.65  0.65
42  586  42  0    860    883   882  0.04  0.39  0.47  0.65  0.65
43  592  43  0    859    882   881  0.04  0.39  0.47  0.65  0.65
44  584  44  0    857    880   880  0.04  0.39  0.47  0.65  0.65
45  521  45  0    856    879   879  0.04  0.39  0.47  0.65  0.65
46  514  46  0    855    878   878  0.04  0.39  0.47  0.65  0.65
47  516  47  0    855    876   876  0.04  0.39  0.47  0.65  0.65
48  515  48  0    854    875   875  0.04  0.39  0.47  0.65  0.65
49  518  49  0    853    875   875  0.04  0.39  0.47  0.67  0.65

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp0.10.050.01-eta10-gam0.01-epoch50/log' mode='wb' closefd=True>


 0 11258 366633.0 write one get article say go know think people like
   0-1 3608 207462.0 god say people one write think article know make believe
     0-1-1 2862 5684.0 militia state arm amendment right party government university professor constitution
     0-1-2 690 193.0 sky translation verse georgia modern disclaimer day catholic desire shoot
     0-1-3 56 0.0 bhj series ram dave knowledge prevent james throw washington legal
   0-2 4369 200414.0 use get write file drive one work problem window know
     0-2-1 4369 3417.0 image tool datum use processing software include plot analysis system
   0-3 2099 83477.0 game team year play go get write player win think
     0-3-1 2077 404.0 double spring sin auto pin wing excellent let block age
     0-3-2 22 2.0 distance bhj series ram dave knowledge prevent james throw washington
   0-4 964 96630.0 use key space system encryption information file chip government new
     0-4-1 964 479.0 palestinian court regulation near radar economic te